# Molecular interaction preferences and the Interaction API

[IsoStar](https://www.ccdc.cam.ac.uk/solutions/csd-core/components/isostar/) is a knowledge-base of intermolecular interaction preferences derived from the CSD and PDB. This data can also be accessed _via_ the [Interaction API](https://downloads.ccdc.cam.ac.uk/documentation/API/descriptive_docs/interaction.html).

In [ ]:
import logging
from pathlib import Path
from platform import platform
import sys
import os
from time import time
import subprocess

import warnings

In [ ]:
import pandas as pd

In [ ]:
with warnings.catch_warnings():
    warnings.filterwarnings(action='ignore', category=DeprecationWarning)  # Ignore current 'distutils Version classes are deprecated' warning
    
    import plotly.express as px

In [ ]:
from IPython.display import HTML

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

In [ ]:
import ccdc
from ccdc.interaction import InteractionLibrary
from ccdc.diagram import DiagramGenerator
from ccdc.io import EntryReader

### Initialization

In [ ]:
logger = logging.getLogger(__name__)

if not logger.hasHandlers():
    handler = logging.StreamHandler()
    handler.setFormatter(logging.Formatter('[%(asctime)s %(levelname)-7s] %(message)s', datefmt='%y-%m-%d %H:%M:%S'))
    logger.addHandler(handler)
    logger.setLevel(logging.INFO)

In [ ]:
logger.info(f"""
Platform:                     {platform()}

Python exe:                   {sys.executable}
Python version:               {'.'.join(str(x) for x in sys.version_info[:3])}

CSD version:                  {ccdc.io.csd_version()}
CSD directory:                {ccdc.io.csd_directory()}
API version:                  {ccdc.__version__}

CSDHOME:                      {os.environ.get('CSDHOME', 'Not set')}
CCDC_LICENSING_CONFIGURATION: {os.environ.get('CCDC_LICENSING_CONFIGURATION', 'Not set')}
""")

Set up a CCDC Diagram Generator...

In [ ]:
diagram_generator = DiagramGenerator()

diagram_generator.settings.return_type = 'SVG'
diagram_generator.settings.explicit_polar_hydrogens = False
diagram_generator.settings.shrink_symbols = False

Utility to help with display in JupyterLab...

In [ ]:
show_df = lambda df: df.style.set_properties(**{'text-align': 'left'})

Initialise the IsoStar central and contact group libraries...

In [ ]:
central_lib = InteractionLibrary.CentralGroupLibrary()
contact_lib = InteractionLibrary.ContactGroupLibrary()

### Inspect the interaction group libraries

Available central groups...

In [ ]:
len(central_lib.groups)

In [ ]:
print('\n'.join(x.name for x in central_lib.groups[:10]))  # First ten names

Available contact groups...

In [ ]:
len(contact_lib.groups)

In [ ]:
print('\n'.join(x.name for x in contact_lib.groups[:10]))   # First ten names

We can visualise the substructure query used to define any group...

In [ ]:
group = central_lib.group_by_name('aromatic-aromatic ester')

HTML(diagram_generator.image(group.substructure_query))

In [ ]:
group = contact_lib.group_by_name('sulfoxide/sulfone O')

HTML(diagram_generator.image(group.substructure_query))

### Perform an Interaction Analysis

We will perform an interaction analyis for aliphatic ketones....

In [ ]:
central_group = central_lib.group_by_name('aliphatic-aliphatic ketone')

HTML(diagram_generator.image(central_group.substructure_query))

The contact groups for which data is available for this central group...

In [ ]:
contact_groups = [x for x in central_group.contact_groups() if x]

len(contact_groups)

We can show the data for these groups as a dataframe...

In [ ]:
def make_row(contact_group):
    
    data = central_group.interaction_data(contact_group)
        
    return [contact_group.name, data.ncontacts, *data.relative_density]

In [ ]:
def depiction(name): # Depiction of a contact-group substruture query
    
    return diagram_generator.image(contact_lib.group_by_name(name).substructure_query)

In [ ]:
contacts_df = (
    pd.DataFrame(
        data=[make_row(x) for x in contact_groups],
        columns=['Contact Group', 'No. of Contacts', 'Relative Density', 'Std. Dev.']
    )
    .sort_values('Relative Density', ascending=False)
    .assign(depiction = lambda df: df['Contact Group'].apply(depiction))
)

contacts_df.shape

In [ ]:
show_df(contacts_df.head(5))

### Inspect which central and contact groups are present in a molecule

Retrieve an example molecule from the CSD (or load one from file)...

In [ ]:
refcode = 'AABHTZ'

with EntryReader('CSD') as reader:
    
    molecule = reader.molecule(refcode)

In [ ]:
HTML(diagram_generator.image(molecule))

Local utility to display a molecule with a substructure highlighted...

In [ ]:
depict_group = lambda mol, group: diagram_generator.image(mol, highlight_atoms=group.match_atoms())

#### Central Groups

Identify the central groups in the molecule of interest...

In [ ]:
central_group_hits = central_lib.search_molecule(molecule)

len(central_group_hits)

In [ ]:
central_groups_df = pd.DataFrame([(group.name, depict_group(molecule, group)) for group in central_group_hits], columns=['Group', 'Depiction'])

show_df(central_groups_df.head(3))

#### Contact Groups

Identify the contact groups in the molecule of interest...

In [ ]:
contact_group_hits = contact_lib.search_molecule(molecule)

len(contact_group_hits)

In [ ]:
contact_groups_df = pd.DataFrame([(group.name, depict_group(molecule, group)) for group in central_group_hits], columns=['Group', 'Depiction'])

show_df(contact_groups_df.head(3))